# League of Conservation Voters - Congressional Votes

https://www.lcv.org/congressional-scorecard/congressional-votes/

In [ ]:
import os
import io
import requests
import zipfile
import tempfile
import shutil
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-election"
s3_prefix = "league_of_conservation_voters/congressional_votes"

In [ ]:
def download_csv(bucket,s3_prefix):
    for year in range(1988,2025):
        print(year)
        # year = 1988
        url=f'https://www.lcv.org/congressional-scorecard/congressional-votes/?export=true&show=scorecard&search=conservation&session_year={year}&issue_filter=all&export-type=votes'
        csv_file=f'lcv_congressional_votes_{year}.csv'
        save_url=f's3://{bucket}/{s3_prefix}/{csv_file}'
        columns = ['vote_title','year','roll_call_vote_num','vote_link']

        t= (con.read_csv(url,delim=',', quote='"',header=False,null_padding=True)
            .mutate(row_id=ibis.row_number())
            .rename(vote_title='column0', 
                    year='column1',
                    num_votes='column2',
                    vote_link='column3'
                   )
           )
        # weird format, splitting up the df into chambers to organize into 1 edf 
        senate_row = t.filter(_.vote_title =="Senate Votes").select('row_id').limit(1).execute()['row_id'].to_list()[0]
        house_row = t.filter(_.vote_title =="House Votes").select('row_id').limit(1).execute()['row_id'].to_list()[0]
        senate_df = t.filter((_.row_id>senate_row+1)&(_.row_id<house_row)).mutate(chamber=ibis.literal('senate'))
        house_df = t.filter((_.row_id>house_row+1)).mutate(chamber=ibis.literal('house'))
        df=senate_df.union(house_df).select('year','chamber','vote_title','num_votes','vote_link').order_by('year','chamber','vote_title')
        
        #only save if there are any votes
        if df.count().execute()>0:
            df.to_csv(save_url) 



In [ ]:
download_csv(bucket,s3_prefix)